<a href="https://colab.research.google.com/github/dead96pool/stress_detection_mlai/blob/main/MLAI_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [1]:
# git repo clone
!git clone https://github.com/dead96pool/datasets
# removing the clone
#!rm -rf datasets

Cloning into 'datasets'...
remote: Enumerating objects: 2817, done.
remote: Counting objects: 100% (1402/1402), done.
remote: Compressing objects: 100% (1388/1388), done.
remote: Total 2817 (delta 10), reused 1401 (delta 9), pack-reused 1415
Receiving objects: 100% (2817/2817), 65.32 MiB | 27.37 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Filtering content: 100% (1357/1357), 853.65 MiB | 19.38 MiB/s, done.


In [65]:
calm_path = "/content/datasets/machine_learning/mlai_project/2019_stress_detection_dataset/CALM/"
stress_path = "/content/datasets/machine_learning/mlai_project/2019_stress_detection_dataset/STRESS/"

In [20]:
column_names = [
    'participant_id', 'age', 'gender', 'key_press_count', 'delete_key_press_count',
    'text_id', 'text_length', 'accelerometer_x', 'accelerometer_y', 'accelerometer_z',
    'accelerometer_magnitude', 'gravity_x', 'gravity_y', 'gravity_z', 'gravity_magnitude',
    'linear_acceleration_x', 'linear_acceleration_y', 'linear_acceleration_z',
    'linear_acceleration_magnitude',
    'gyroscope_x', 'gyroscope_y', 'gyroscope_z', 'gyroscope_magnitude',
    'magnetometer_x', 'magnetometer_y', 'magnetometer_z', 'magnetometer_magnitude',
    'light_sensor', 'proximity_sensor',
    'game_rotation_vector_x', 'game_rotation_vector_y', 'game_rotation_vector_z',
    'game_rotation_vector_magnitude',
    'rotation_sensor_x', 'rotation_sensor_y', 'rotation_sensor_z', 'rotation_sensor_scalar',
    'rotation_sensor_value',
    'rotation_sensor_magnitude',
    'orientation_azimuth', 'orientation_pitch', 'orientation_roll', 'orientation_sensor_magnitude',
    'number_of_steps'
]

In [66]:
# file names are in the format : CALMorSTRESS_{participant_id}_{text_id}
#file_1 = "CALM_1_1.csv"
#file_2 = "CALM_1_2.csv"
#calm_df_1 = pd.read_csv(calm_path+file_1, sep=';', names=column_names)
#calm_df_2 = pd.read_csv(calm_path+file_2, sep=';', names=column_names)

In [128]:
# looping throught the calm folder to combine all the CALM files
calm_pattern = calm_path + "CALM_*.csv"
calm_files = glob.glob(calm_pattern)

calm_df = []

for file in calm_files:
    df = pd.read_csv(file, sep=';', names=column_names, usecols=range(44))
    calm_df.append(df)


In [131]:
# looping throught the calm folder to combine all the STRESS files
stress_pattern = stress_path + "STRESS_*.csv"
stress_files = glob.glob(stress_pattern)

stress_df = []

for file in calm_files:
    df = pd.read_csv(file, sep=';', names=column_names, usecols=range(44))
    stress_df.append(df)


In [129]:

# combining all the calm files
calm_combine_df = pd.concat(calm_df, axis=0, ignore_index= True)

In [132]:

# combining all the stress files
stress_combine_df = pd.concat(stress_df, axis=0, ignore_index= True)

In [146]:
calm_combine_df["stress"] = 0
stress_combine_df["stress"] = 1

In [147]:

display(calm_combine_df.sort_values(by='participant_id'))

,participant_id,age,gender,key_press_count,delete_key_press_count,text_id,text_length,accelerometer_x,accelerometer_y,accelerometer_z,...,rotation_sensor_z,rotation_sensor_scalar,rotation_sensor_value,rotation_sensor_magnitude,orientation_azimuth,orientation_pitch,orientation_roll,orientation_sensor_magnitude,number_of_steps,stress
109254,1,30,Male,10,0,5,75,-1.302760,4.837598,8.425615,...,-0.300898,0.916395,0.174533,1.015117,34.401306,-29.803370,-7.332992,46.102750,14.0,0
595675,1,30,Male,17,0,4,100,-0.626676,4.450880,8.777250,...,-0.301260,0.922875,0.174533,1.015117,35.075410,-27.598997,-4.566661,44.864723,14.0,0
595674,1,30,Male,17,0,4,100,-1.039916,4.528765,8.836761,...,-0.301465,0.922398,0.174533,1.015117,35.075410,-27.598997,-4.566661,44.864723,14.0,0
595673,1,30,Male,17,0,4,100,-1.039916,4.528765,8.836761,...,-0.301465,0.922398,0.174533,1.015117,35.075410,-27.598997,-4.566661,44.864723,14.0,0
595672,1,30,Male,17,0,4,100,-1.137322,4.552945,8.814731,...,-0.299603,0.922396,0.174533,1.015117,34.717102,-27.784573,-5.137813,44.762226,14.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683894,110,21,Male,52,4,3,69,1.237840,8.370385,5.063891,...,0.586636,0.611730,0.000000,0.999999,271.424000,-63.966354,2.785785,278.873540,0.0,0
683893,110,21,Male,52,4,3,69,0.000000,9.349643,3.993650,...,0.586890,0.608465,0.000000,0.999999,271.261840,-64.477640,2.294088,278.829000,0.0,0
683892,110,21,Male,52,4,3,69,0.000000,9.349643,3.993650,...,0.586890,0.608465,0.000000,0.999999,271.261840,-64.477640,2.294088,278.829000,0.0,0
683898,110,21,Male,53,4,3,69,0.675185,8.918674,4.668836,...,0.585964,0.611677,0.000000,0.999999,271.526520,-64.086000,2.664012,278.999570,0.0,0


In [148]:
display(stress_combine_df.sort_values(by='participant_id'))

,participant_id,age,gender,key_press_count,delete_key_press_count,text_id,text_length,accelerometer_x,accelerometer_y,accelerometer_z,...,rotation_sensor_z,rotation_sensor_scalar,rotation_sensor_value,rotation_sensor_magnitude,orientation_azimuth,orientation_pitch,orientation_roll,orientation_sensor_magnitude,number_of_steps,stress
109254,1,30,Male,10,0,5,75,-1.302760,4.837598,8.425615,...,-0.300898,0.916395,0.174533,1.015117,34.401306,-29.803370,-7.332992,46.102750,14.0,1
595675,1,30,Male,17,0,4,100,-0.626676,4.450880,8.777250,...,-0.301260,0.922875,0.174533,1.015117,35.075410,-27.598997,-4.566661,44.864723,14.0,1
595674,1,30,Male,17,0,4,100,-1.039916,4.528765,8.836761,...,-0.301465,0.922398,0.174533,1.015117,35.075410,-27.598997,-4.566661,44.864723,14.0,1
595673,1,30,Male,17,0,4,100,-1.039916,4.528765,8.836761,...,-0.301465,0.922398,0.174533,1.015117,35.075410,-27.598997,-4.566661,44.864723,14.0,1
595672,1,30,Male,17,0,4,100,-1.137322,4.552945,8.814731,...,-0.299603,0.922396,0.174533,1.015117,34.717102,-27.784573,-5.137813,44.762226,14.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683894,110,21,Male,52,4,3,69,1.237840,8.370385,5.063891,...,0.586636,0.611730,0.000000,0.999999,271.424000,-63.966354,2.785785,278.873540,0.0,1
683893,110,21,Male,52,4,3,69,0.000000,9.349643,3.993650,...,0.586890,0.608465,0.000000,0.999999,271.261840,-64.477640,2.294088,278.829000,0.0,1
683892,110,21,Male,52,4,3,69,0.000000,9.349643,3.993650,...,0.586890,0.608465,0.000000,0.999999,271.261840,-64.477640,2.294088,278.829000,0.0,1
683898,110,21,Male,53,4,3,69,0.675185,8.918674,4.668836,...,0.585964,0.611677,0.000000,0.999999,271.526520,-64.086000,2.664012,278.999570,0.0,1


In [104]:
print(calm_combine_df.isnull().sum())
print(calm_combine_df.info())

participant_id                    0
age                               0
gender                            0
key_press_count                   0
delete_key_press_count            0
text_id                           0
text_length                       0
accelerometer_x                   0
accelerometer_y                   0
accelerometer_z                   0
accelerometer_magnitude           0
gravity_x                         0
gravity_y                         0
gravity_z                         0
gravity_magnitude                 0
linear_acceleration_x             0
linear_acceleration_y             0
linear_acceleration_z             0
linear_acceleration_magnitude     0
gyroscope_x                       0
gyroscope_y                       0
gyroscope_z                       0
gyroscope_magnitude               0
magnetometer_x                    0
magnetometer_y                    0
magnetometer_z                    0
magnetometer_magnitude            0
light_sensor                

In [149]:
# finding unique participant values
print(sorted(calm_combine_df["participant_id"].unique()))
print(len(calm_combine_df["participant_id"].unique()))

print(sorted(stress_combine_df["participant_id"].unique()))
print(len(stress_combine_df["participant_id"].unique()))

print(sorted(calm_combine_df["stress"].unique()))
print(len(calm_combine_df["stress"].unique()))

print(sorted(stress_combine_df["stress"].unique()))
print(len(stress_combine_df["stress"].unique()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110]
110
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110]
110
[0]
1
[1]
1


In [135]:
print(stress_combine_df.isnull().sum())
print(stress_combine_df.info())

participant_id                    0
age                               0
gender                            0
key_press_count                   0
delete_key_press_count            0
text_id                           0
text_length                       0
accelerometer_x                   0
accelerometer_y                   0
accelerometer_z                   0
accelerometer_magnitude           0
gravity_x                         0
gravity_y                         0
gravity_z                         0
gravity_magnitude                 0
linear_acceleration_x             0
linear_acceleration_y             0
linear_acceleration_z             0
linear_acceleration_magnitude     0
gyroscope_x                       0
gyroscope_y                       0
gyroscope_z                       0
gyroscope_magnitude               0
magnetometer_x                    0
magnetometer_y                    0
magnetometer_z                    0
magnetometer_magnitude            0
light_sensor                

In [150]:
# combining the stress and the calm dataframes together

combined_df = pd.concat([calm_combine_df, stress_combine_df], ignore_index = True, axis = 0)

In [152]:
display(combined_df.sort_values(by="participant_id"))

,participant_id,age,gender,key_press_count,delete_key_press_count,text_id,text_length,accelerometer_x,accelerometer_y,accelerometer_z,...,rotation_sensor_z,rotation_sensor_scalar,rotation_sensor_value,rotation_sensor_magnitude,orientation_azimuth,orientation_pitch,orientation_roll,orientation_sensor_magnitude,number_of_steps,stress
601961,1,30,Male,21,0,2,47,-1.872126,5.606165,7.378550,...,-0.293177,0.902049,0.174533,1.015117,32.76526,-35.328617,-10.556442,49.326584,14.0,0
1127681,1,30,Male,10,0,3,66,-1.187407,4.757959,8.627420,...,-0.296549,0.918059,0.174533,1.015117,34.14560,-29.933722,-6.193084,45.829075,14.0,1
1127682,1,30,Male,10,0,3,66,-1.187407,4.757959,8.627420,...,-0.298849,0.918205,0.174533,1.015117,34.53354,-29.633608,-5.755253,45.867626,14.0,1
1127683,1,30,Male,10,0,3,66,-1.187407,4.757959,8.627420,...,-0.298849,0.918205,0.174533,1.015117,34.53354,-29.633608,-5.755253,45.867626,14.0,1
1127684,1,30,Male,10,0,3,66,-1.187407,4.757959,8.627420,...,-0.298849,0.918205,0.174533,1.015117,34.53354,-29.633608,-5.755253,45.867626,14.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87123,110,21,Male,91,10,7,71,0.244216,6.689604,7.285778,...,0.746642,0.559984,0.000000,0.999999,253.33801,-42.060238,1.688863,256.811340,0.0,0
87124,110,21,Male,91,10,7,71,0.244216,6.689604,7.285778,...,0.746642,0.559984,0.000000,0.999999,253.33801,-42.060238,1.688863,256.811340,0.0,0
87125,110,21,Male,91,10,7,71,0.239427,6.701575,7.374366,...,0.746721,0.560213,0.000000,0.999999,253.36621,-41.997730,1.640403,256.828600,0.0,0
87112,110,21,Male,90,10,7,71,0.067040,6.912272,7.379155,...,0.747235,0.554940,0.000000,0.999999,252.84040,-42.870426,1.507553,256.453520,0.0,0


In [155]:
# pearse correlation matrix
correlation_matrix = combined_df.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")

# Add title
plt.title('Pearson Correlation Matrix')

# Show plot
plt.show()

ValueError: could not convert string to float: 'Male'